In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker

from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor


In [2]:
# IMPORT CSV FILES 

# -- PER COUNTRY FEATURES --
rank = pd.read_csv("Features/fifa_rank.csv")
player_avg = pd.read_csv("Features/player_averages_for_teams.csv")
win_ratio = pd.read_csv("Features/ratio_played_vs_won.csv")
delta_pts = pd.read_csv("Features/total_score_margins.csv")
games = pd.read_csv("Features/adv_df_pivot.csv")
player_top_features = pd.read_csv("Features/player_top_features_country.csv")

# -- OTHER FEATURES --
df_matches = pd.read_csv("Features/wc_matches.csv")
schedule = pd.read_csv('Data/schedule.csv')

# IDENTIFY 2022 WORLD CUP TEAMS
teams_2022 = ['Qatar', 'Netherlands', 'Senegal', 'Ecuador', 
            'England', 'United States', 'Wales', 'Iran', 
            'Argentina', 'Poland', 'Mexico', 'Saudi Arabia', 
            'France', 'Denmark', 'Tunisia', 'Australia', 
            'Germany', 'Spain', 'Japan', 'Costa Rica', 
            'Belgium', 'Croatia', 'Canada', 'Morocco', 
            'Brazil', 'Switzerland', 'Serbia', 'Cameroon', 
            'Portugal', 'Uruguay', 'Ghana', 'Korea Republic']

In [3]:
# matches_by_year = df_matches[["date", 'year']].groupby(["year"]).count()
# matches_by_year.head(50)

In [4]:
# CLEAN UP

rank = rank.drop(['Unnamed: 0', 'Points'], axis=1)
player_avg = player_avg.rename(columns={"Nationality": "Country"})
win_ratio = win_ratio.drop(['Unnamed: 0'], axis=1)
delta_pts = delta_pts.drop(['Unnamed: 0'], axis=1)
games = games.rename(columns={"country": "Country"})
player_top_features = player_top_features.drop(['Unnamed: 0'], axis=1)

In [5]:
# MERGE DATAFRAMES ON "COUNTRY"

 # list of dataframes
# dfs = [rank, player_avg, win_ratio, delta_pts, games]
dfs = [rank, player_top_features, win_ratio, delta_pts, games]

# MERGING
compiled_df = reduce(lambda  left,right: pd.merge(left,right,on=["Country"],
                                            how='outer'), dfs)
# FILLING NULLS WITH 0
compiled_df = compiled_df.fillna(0)
compiled_df["Country"].unique()

array(['Brazil', 'Belgium', 'Argentina', 'France', 'England', 'Spain',
       'Netherlands', 'Portugal', 'Denmark', 'Germany', 'Mexico',
       'Uruguay', 'United States', 'Croatia', 'Switzerland', 'Senegal',
       'Wales', 'Morocco', 'Iran', 'Japan', 'Serbia', 'Poland',
       'Korea Republic', 'Tunisia', 'Costa Rica', 'Cameroon', 'Australia',
       'Canada', 'Ecuador', 'Qatar', 'Saudi Arabia', 'Ghana'],
      dtype=object)

In [6]:
# limit the dataframe rows to the relevant teams playing in the 2022 world cup
compiled_df = compiled_df[(compiled_df['Country'].isin(teams_2022))]
compiled_df.head()

Rank    Country  avg_Composure  avg_Overall  avg_Potential  avg_Volleys  \
0     1     Brazil        64.8362      70.9672        73.1028      49.5706   
1     2    Belgium        60.6567      67.6767        73.9033      44.6500   
2     3  Argentina        60.8926      67.8063        72.7284      45.2558   
3     4     France        60.0427      67.2386        73.5875      43.5765   
4     5    England        56.4206      63.2728        70.3153      40.3194   

   avg_Reactions  total_wins  total_games  ratio_won  win_margins  \
0        68.1390        56.0         77.0   0.727273     3.790698   
1        62.5800        21.0         42.0   0.500000     2.500000   
2        63.9053        39.0         65.0   0.600000     2.823529   
3        62.4125        29.0         50.0   0.580000     3.576190   
4        59.0320        22.0         30.0   0.733333     3.232143   

   lose_margins  final  group stage  quarter-finals  round of 16  \
0     -1.800000    4.0         39.0             9.0          9.0   
1     -1.903846    0.0         27.0             3.0          6.0   
2     -2.059524    4.0         36.0             6.0          8.0   
3     -1.400000    3.0         27.0             5.0          5.0   
4     -1.064706    0.0         30.0             6.0          7.0   

   second group stage  semi-finals  third-place match  
0                 8.0          5.0                3.0  
1                 2.0          2.0                2.0  
2                 8.0          3.0                0.0  
3                 2.0          5.0                2.0  
4                 2.0          2.0                2.0

In [7]:
df_matches.head(15)
df_matches = df_matches.replace("South Korea", "Korea Republic")

# df_matches["home_team"].unique()


In [8]:
df_matches_home = df_matches[df_matches['home_team'].isin(teams_2022)]
df_matches_away = df_matches[df_matches['away_team'].isin(teams_2022)]
df_matches2 = pd.concat((df_matches_home, df_matches_away))
df_matches2.drop_duplicates()
df_matches2.count()

Unnamed: 0         5150
date               5150
home_team          5150
away_team          5150
home_score         5150
away_score         5150
tournament         5150
city               5150
country            5150
neutral            5150
winning_team       5150
goal_difference    5150
year               5150
dtype: int64

In [9]:
df_matches2 = df_matches2.drop(columns = "Unnamed: 0", axis = 1)

df_matches2.head()

date  home_team      away_team  home_score  away_score  \
0  1930-07-13    Belgium  United States           0           3   
1  1930-07-13     France         Mexico           4           1   
2  1930-07-14     Brazil     Yugoslavia           1           2   
3  1930-07-15  Argentina         France           1           0   
6  1930-07-18    Uruguay           Peru           1           0   

       tournament        city  country  neutral   winning_team  \
0  FIFA World Cup  Montevideo  Uruguay     True  United States   
1  FIFA World Cup  Montevideo  Uruguay     True         France   
2  FIFA World Cup  Montevideo  Uruguay     True     Yugoslavia   
3  FIFA World Cup  Montevideo  Uruguay     True      Argentina   
6  FIFA World Cup  Montevideo  Uruguay    False        Uruguay   

   goal_difference  year  
0                3  1930  
1                3  1930  
2                1  1930  
3                1  1930  
6                1  1930

In [10]:
df_matches2 = df_matches2.drop(["date", "home_score", "away_score", "tournament", "city", "country", "goal_difference", "neutral", "year"], axis = 1)
df_matches2.head()

home_team      away_team   winning_team
0    Belgium  United States  United States
1     France         Mexico         France
2     Brazil     Yugoslavia     Yugoslavia
3  Argentina         France      Argentina
6    Uruguay           Peru        Uruguay

In [11]:
#Building the model
#the prediction label: 
# The winning_team column will show "2" if the home team has won, "1" if it was a tie, and "0" if the away team has won.

# df_matches2 = df_matches2.reset_index(drop=True)
df_matches2.loc[df_matches2.winning_team == df_matches2.home_team,'winning_team']=2
df_matches2.loc[df_matches2.winning_team == 'draw', 'winning_team']=1
df_matches2.loc[df_matches2.winning_team == df_matches2.away_team, 'winning_team']=0

df_matches2.head()

home_team      away_team winning_team
0    Belgium  United States            0
1     France         Mexico            2
2     Brazil     Yugoslavia            0
3  Argentina         France            2
6    Uruguay           Peru            2

## ADD IN COUNTRY FEATURES HERE (PROBABLY)

In [12]:
all_feautres_home = pd.merge(df_matches2, compiled_df, left_on  = "home_team", right_on = "Country", how ="left" )
all_feautres_away = pd.merge(df_matches2, compiled_df, left_on  = "away_team", right_on = "Country", how ="left" )

# all_feautres_away
# all_feautres_home

all_features = all_feautres_home.append(all_feautres_away, ignore_index = True)
all_features = all_features.dropna()
all_features["Country"].nunique()

32

In [13]:
final = pd.get_dummies(all_features, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6842 entries, 0 to 10299
Columns: 403 entries, winning_team to away_team_Zimbabwe
dtypes: float64(18), object(2), uint8(383)
memory usage: 3.6+ MB


In [14]:
#convert home team and away team from categorical variables to continous inputs 
# Get dummy variables
# final = pd.get_dummies(df_matches2, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

# Separate X and y sets
X = final.drop(['winning_team', "Country"], axis=1)
y = final["winning_team"]
y = y.astype('int')

# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [15]:
final.info()
y.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6842 entries, 0 to 10299
Columns: 403 entries, winning_team to away_team_Zimbabwe
dtypes: float64(18), object(2), uint8(383)
memory usage: 3.6+ MB


2    3372
0    1834
1    1636
Name: winning_team, dtype: int64

In [16]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

scaler = preprocessing.StandardScaler().fit(X_train)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
score = logreg.score(X_train_scaled, y_train)
score2 = logreg.score(X_test_scaled, y_test)

# from sklearn.ensemble import RandomForestClassifier


# clf = RandomForestClassifier(random_state=37, n_estimators=1000).fit(X_train_scaled, y_train)

# score = clf.score(X_train, y_train)
# score2 = clf.score(X_test, y_test)

print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.549
Test set accuracy:  0.512


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
#adding Fifa rankings
#the team which is positioned higher on the FIFA Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "home_teams" column
#since there are no "home" or "away" teams in World Cup games. 

# Loading new datasets
# ranking = pd.read_csv('datasets/fifa_rankings.csv') 
fixtures = pd.read_csv('Data/schedule.csv')

# List for storing the group stage games
pred_set = []

## Add country  features here ?

In [19]:
# from github repo--------- itsmuriuki

# #adding Fifa rankings
# #the team which is positioned higher on the FIFA Ranking will be considered "favourite" for the match
# #and therefore, will be positioned under the "home_teams" column
# #since there are no "home" or "away" teams in World Cup games. 

# # Loading new datasets
# ranking = pd.read_csv('datasets/fifa_rankings.csv') 
# fixtures = pd.read_csv('datasets/fixtures.csv')

# # List for storing the group stage games
# pred_set = []

# loading datasets
country_features = compiled_df
fixtures = schedule

# list for storing group stage games
pred_set = []



In [20]:
# may not need the .inster and .map functions as we're trying to use more than one feature
# cannot not specify a single columns as currently written

# ------
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Home Team'].map(country_features .set_index('Country')['avg_Composure']))
fixtures.insert(2, 'second_position', fixtures['Away Team'].map(country_features .set_index('Country')['avg_Composure']))

# fixtures.insert(1, 'first_position', fixtures['Home Team'].map(country_features.set_index('Country')))
# fixtures.insert(2, 'second_position', fixtures['Away Team'].map(country_features.set_index('Country')))


# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:48, :]
fixtures.tail()

Match Number  first_position  second_position Round Number  \
43            44         61.9655          56.1481            3   
44            45         60.9836          58.3902            3   
45            46         55.5819          64.4681            3   
46            47         62.2406          57.6938            3   
47            48         60.5294          64.8362            3   

               Date                Location       Home Team    Away Team  \
43  1/12/2022 19:00         Al Bayt Stadium      Costa Rica      Germany   
44  2/12/2022 15:00       Al Janoub Stadium           Ghana      Uruguay   
45  2/12/2022 15:00  Education City Stadium  Korea Republic     Portugal   
46  2/12/2022 19:00             Stadium 974          Serbia  Switzerland   
47  2/12/2022 19:00          Lusail Stadium        Cameroon       Brazil   

      Group  Result  
43  Group E     NaN  
44  Group H     NaN  
45  Group H     NaN  
46  Group G     NaN  
47  Group G     NaN

In [21]:

# Loop to add teams to new prediction dataset based on the ranking position of each team
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'home_team': row['Home Team'], 'away_team': row['Away Team'], 'winning_team': None})
    else:
        pred_set.append({'home_team': row['Away Team'], 'away_team': row['Home Team'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set

pred_set.head()

home_team  away_team winning_team
0   Netherlands    Senegal         None
1       England       Iran         None
2         Qatar    Ecuador         None
3         Wales        USA         None
4  Saudi Arabia  Argentina         None

In [22]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['home_team', 'away_team'], columns=['home_team', 'away_team'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]

# Remove winning team column
pred_set = pred_set.drop(['winning_team', "Country"], axis=1)

pred_set.head()

C:\Users\madis\AppData\Local\Temp/ipykernel_14332/2413084969.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pred_set[c] = 0


Rank  avg_Composure  avg_Overall  avg_Potential  avg_Volleys  \
0     0              0            0              0            0   
1     0              0            0              0            0   
2     0              0            0              0            0   
3     0              0            0              0            0   
4     0              0            0              0            0   

   avg_Reactions  total_wins  total_games  ratio_won  win_margins  ...  \
0              0           0            0          0            0  ...   
1              0           0            0          0            0  ...   
2              0           0            0          0            0  ...   
3              0           0            0          0            0  ...   
4              0           0            0          0            0  ...   

   away_team_Uruguay  away_team_Uzbekistan  away_team_Vanuatu  \
0                  0                     0                  0   
1                  0                     0                  0   
2                  0                     0                  0   
3                  0                     0                  0   
4                  0                     0                  0   

   away_team_Venezuela  away_team_Vietnam  away_team_Wales  away_team_Yemen  \
0                    0                  0                0                0   
1                    0                  0                0                0   
2                    0                  0                0                0   
3                    0                  0                0                0   
4                    0                  0                0                0   

   away_team_Yugoslavia  away_team_Zambia  away_team_Zimbabwe  
0                     0                 0                   0  
1                     0                 0                   0  
2                     0                 0                   0  
3                     0                 0                   0  
4                     0                 0                   0  

[5 rows x 401 columns]

In [23]:
fixtures2 = fixtures.drop(columns = ["first_position", "second_position", "Date", "Location", "Result"], axis =1)
# fixtures2
fixtures2

Match Number Round Number       Home Team       Away Team    Group
0              2            1         Senegal     Netherlands  Group A
1              3            1         England            Iran  Group B
2              1            1           Qatar         Ecuador  Group A
3              4            1             USA           Wales  Group B
4              8            1       Argentina    Saudi Arabia  Group C
5              6            1         Denmark         Tunisia  Group D
6              7            1          Mexico          Poland  Group C
7              5            1          France       Australia  Group D
8             12            1         Morocco         Croatia  Group F
9             11            1         Germany           Japan  Group E
10            10            1           Spain      Costa Rica  Group E
11             9            1         Belgium          Canada  Group F
12            13            1     Switzerland        Cameroon  Group G
13            14            1         Uruguay  Korea Republic  Group H
14            15            1        Portugal           Ghana  Group H
15            16            1          Brazil          Serbia  Group G
16            17            2           Wales            Iran  Group B
17            18            2           Qatar         Senegal  Group A
18            19            2     Netherlands         Ecuador  Group A
19            20            2         England             USA  Group B
20            21            2         Tunisia       Australia  Group D
21            22            2          Poland    Saudi Arabia  Group C
22            23            2          France         Denmark  Group D
23            24            2       Argentina          Mexico  Group C
24            25            2           Japan      Costa Rica  Group E
25            26            2         Belgium         Morocco  Group F
26            27            2         Croatia          Canada  Group F
27            28            2           Spain         Germany  Group E
28            29            2        Cameroon          Serbia  Group G
29            30            2  Korea Republic           Ghana  Group H
30            31            2          Brazil     Switzerland  Group G
31            32            2        Portugal         Uruguay  Group H
32            35            3         Ecuador         Senegal  Group A
33            36            3     Netherlands           Qatar  Group A
34            33            3           Wales         England  Group B
35            34            3            Iran             USA  Group B
36            37            3       Australia         Denmark  Group D
37            38            3         Tunisia          France  Group D
38            39            3          Poland       Argentina  Group C
39            40            3    Saudi Arabia          Mexico  Group C
40            41            3         Croatia         Belgium  Group F
41            42            3          Canada         Morocco  Group F
42            43            3           Japan           Spain  Group E
43            44            3      Costa Rica         Germany  Group E
44            45            3           Ghana         Uruguay  Group H
45            46            3  Korea Republic        Portugal  Group H
46            47            3          Serbia     Switzerland  Group G
47            48            3        Cameroon          Brazil  Group G

In [24]:
#group matches 
predictions = logreg.predict(pred_set)

group_16_win = []
group_16_draw = []
group_16_loss = []
group_16_scores = []
# FirstTeam_prob", "Draw", "SecondTeam_prob"])
match_name = pd.Series([])
fixtures2["Winner"] = pd.Series([])

for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 2:
        print("Winner: " + backup_pred_set.iloc[i, 1])
        if logreg.predict_proba(pred_set)[i][2] > logreg.predict_proba(pred_set)[i][0]:
            group_16_scores.append(2)
        
    elif predictions[i] == 1:
        print("Draw")
        group_16_draw.append(backup_pred_set.iloc[i, 1])
        group_16_draw.append(backup_pred_set.iloc[i, 0])

        if logreg.predict_proba(pred_set)[i][1] > logreg.predict_proba(pred_set)[i][2] or logreg.predict_proba(pred_set)[i][1] > logreg.predict_proba(pred_set)[i][0]:
            group_16_scores.append(1)

    elif predictions[i] == 0:
        print("Winner: " + backup_pred_set.iloc[i, 0])
        group_16_loss.append(backup_pred_set.iloc[i, 0])
        
        if logreg.predict_proba(pred_set)[i][0] > logreg.predict_proba(pred_set)[i][2]:
            group_16_scores.append(0)
            
#     FIRST TEAM
    print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set)[i][2]))
#     DRAW
    print('Probability of Draw: ', '%.3f'%(logreg.predict_proba(pred_set)[i][1]))
#     SECOND TEAM
    print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set)[i][0]))
    print("")
    

C:\Users\madis\AppData\Local\Temp/ipykernel_14332/4232133660.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  match_name = pd.Series([])
C:\Users\madis\AppData\Local\Temp/ipykernel_14332/4232133660.py:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fixtures2["Winner"] = pd.Series([])


Senegal and Netherlands
Winner: Senegal
Probability of Senegal winning:  0.355
Probability of Draw:  0.341
Probability of Netherlands winning:  0.304

Iran and England
Winner: Iran
Probability of Iran winning:  0.345
Probability of Draw:  0.319
Probability of England winning:  0.336

Ecuador and Qatar
Winner: Ecuador
Probability of Ecuador winning:  0.366
Probability of Draw:  0.309
Probability of Qatar winning:  0.325

USA and Wales
Winner: Wales
Probability of USA winning:  0.313
Probability of Draw:  0.319
Probability of Wales winning:  0.368

Argentina and Saudi Arabia
Draw
Probability of Argentina winning:  0.329
Probability of Draw:  0.340
Probability of Saudi Arabia winning:  0.331

Tunisia and Denmark
Draw
Probability of Tunisia winning:  0.325
Probability of Draw:  0.346
Probability of Denmark winning:  0.329

Mexico and Poland
Draw
Probability of Mexico winning:  0.323
Probability of Draw:  0.349
Probability of Poland winning:  0.328

France and Australia
Draw
Probability of 

In [25]:
# group_16_win_s = pd.Series(group_16_win)
# group_16_draw_s = pd.Series(group_16_draw)
# group_16_loss_s = pd.Series(group_16_loss)

# # group_16_scores = pd.Series(group_16_scores, dtype="Int64")
# # group_16_df = pd.concat([group_16_win, group_16_draw,group_16_loss, group_16_scores])

# # group_16_df
# # print(group_16_draw_s)
# backup_pred_set["outcomes_homewin"] = backup_pred_set["home_team"].isin(group_16_win_s)
# backup_pred_set["outcomes_homeloss"] = backup_pred_set["home_team"].isin(group_16_loss_s)

# # backup_pred_set["outcomes_awaywin"] = backup_pred_set["home_team"].isin(group_16_win_s)
# # backup_pred_set["outcomes_awayloss"] = backup_pred_set["home_team"].isin(group_16_win_s)

# backup_pred_set["outcomes_d"] = backup_pred_set["home_team"].isin(group_16_draw_s)




# backup_pred_set

In [26]:
group_16_scores_s = pd.Series(group_16_scores, name = "outcomes")

fixtures2= fixtures2.merge(group_16_scores_s, left_index=True, right_index=True)


In [27]:

fixtures2.sort_values("Home Team")
fixtures2["outcomes_away"] = np.where(fixtures2["outcomes"] ==2, 0,np.where(fixtures2["outcomes"] ==1, 1,np.where(fixtures2["outcomes"] ==0, 2, "")))

fixtures3_h = fixtures2.drop(columns = "Away Team", axis = 1)
fixtures3_h = fixtures2.groupby("Home Team").sum("outcomes")

fixtures3_a = fixtures2.drop(columns = "Home Team", axis = 1)
fixtures3_a = fixtures2.groupby("Away Team").sum("outcomes_away")      
fixtures3_h.head()

Match Number  Winner  outcomes
Home Team                                
Argentina            32     0.0         3
Australia            37     0.0         2
Belgium              35     0.0         2
Brazil               47     0.0         0
Cameroon             77     0.0         0

In [28]:
fixtures3_a.head()

Match Number  Winner  outcomes
Away Team                                
Argentina            39     0.0         1
Australia            26     0.0         2
Belgium              41     0.0         0
Brazil               48     0.0         0
Cameroon             13     0.0         0